In [1]:
from DataIO import DataIO, City, Service
from datetime import date

In [2]:
day = date(2019, 4, 15)
data = DataIO.load_file(city=City.BORDEAUX, service=Service.TWITCH, day=day, uplink=False)
data.head()

,tile_id,00:00,00:15,00:30,00:45,01:00,01:15,01:30,01:45,02:00,...,21:30,21:45,22:00,22:15,22:30,22:45,23:00,23:15,23:30,23:45
0,248,2522,987,385,187,238,170,202,189,134,...,202,335,4765,306,412,391,347,275,183,134
1,587,3232,1225,429,143,81,71,136,75,56,...,204,231,6147,241,379,311,281,139,112,60
2,588,3142,1186,409,130,53,58,104,56,53,...,142,152,5865,133,115,117,179,128,104,59
3,589,2949,1119,398,147,109,114,156,117,108,...,65,121,5384,113,176,184,211,204,142,120
4,590,2544,988,375,167,210,158,201,163,123,...,148,221,4665,184,256,261,280,220,164,135
